In [ ]:
!pip install allennlp allennlp-models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 730 kB 30.4 MB/s 
     |████████████████████████████████| 464 kB 73.6 MB/s 
     |████████████████████████████████| 1.8 MB 54.8 MB/s 
     |████████████████████████████████| 182 kB 73.7 MB/s 
     |████████████████████████████████| 248 kB 72.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 316 kB 78.7 MB/s 
     |████████████████████████████████| 125 kB 82.6 MB/s 
     |████████████████████████████████| 6.5 MB 49.4 MB/s 
     |████████████████████████████████| 880 kB 75.2 MB/s 
     |████████████████████████████████| 107 kB 79.0 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 305 kB 78.5 MB/s 
     |████████████████████████████████

In [ ]:
!pip install git+https://github.com/explosion/spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/explosion/spacy-transformers to /tmp/pip-req-build-45p0a0mt
  Running command git clone -q https://github.com/explosion/spacy-transformers /tmp/pip-req-build-45p0a0mt
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1 MB 30.2 MB/s 
     |████████████████████████████████| 6.6 MB 60.8 MB/s 
     |████████████████████████████████| 819 kB 60.3 MB/s 
  Created wheel for spacy-transformers: filename=spacy_transformers-1.2.0.dev0-cp38-cp38-linux_x86_64.whl size=580962 sha256=a92b4e3d3b120d6c268c7d6fa04ea30f612c4bfd215c3b541f970ce6088c68c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-6oaaacm4/wheels/a7/ee/1d/091063e29f946921f6d471003bc743f2ada4e4bafa62f0a0d1
Successfully built spacy-transformers
  Attemp

In [ ]:
!pip install -q wikipedia

In [ ]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
import json
from multiprocessing.dummy import Pool as ThreadPool
import tqdm
import unicodedata
import wikipedia

In [ ]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/elmo-constituency-parser-2020.02.10.tar.gz")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Output()

Output()

Output()

In [25]:
claim1 = "Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League."
parse=predictor.predict(
    sentence=claim1
)
root = parse['hierplane_tree']['root']

In [ ]:
def extract_noun_phrases(root):
	np_list = []
	stack = [root]
	while len(stack):
		top = stack.pop()
		if top['nodeType'] == 'NP':
			np_list.append(top['word'])

		if 'children' in top.keys():
			for child in top['children']:
				stack.append(child)
	return np_list

In [ ]:
noun_phrases = extract_noun_phrases(root)
noun_phrases

['the 49ers 63rd season in the National Football League',
 'the National Football League',
 'the 49ers 63rd season',
 'a starting quarterback',
 'Colin Kaepernick']

In [ ]:
def extract_other_entities(root, claim):
	other_entities = []
	words = []
	claim_tokens = word_tokenize(claim) 
	for word in claim_tokens:
		nodeType = find_word(root, word)
		if nodeType == 'VB' or nodeType == 'ADVP' or nodeType == 'VP' or nodeType == 'VBD':
			if len(words):
				other_entities.append(' '.join(words))
			words.append(word)
		else:
			words.append(word)
	return other_entities

def find_word(root, word):
	stack = [root]
	while len(stack):
		top = stack.pop()
		if top['word'] == word:
			return top['nodeType']
		if 'children' in top.keys():
			for child in top['children']:
				stack.append(child)
	return None

In [ ]:
other_entities = extract_other_entities(root, claim1)
other_entities

['Colin Kaepernick']

In [ ]:
entities = []
entities.extend(noun_phrases)
entities.extend(other_entities)
entities.append(claim1)
entities = list(set(entities))
entities

['Colin Kaepernick',
 'Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League.',
 'the 49ers 63rd season',
 'the National Football League',
 'the 49ers 63rd season in the National Football League',
 'a starting quarterback']

In [ ]:
def wiki_search(claim, entities):
	k = 7
	documents = []
	for entity in entities:
		try:
			if entity is not None:
				results = wikipedia.search(entity)
				documents.extend(results[:k])
		except:
			print(entity)
			print(claim)

	return list(set(documents))

In [ ]:
documents = wiki_search(claim1, entities)
documents

['2012 San Francisco 49ers season',
 'Lists of National Football League team seasons',
 'List of New York Jets starting quarterbacks',
 'National Football League Most Valuable Player Award',
 'Steve Young',
 '2005 San Francisco 49ers season',
 'List of defunct National Football League franchises',
 '2020 San Francisco 49ers season',
 'History of the National Football League',
 'List of Detroit Lions starting quarterbacks',
 '2022 San Francisco 49ers season',
 'List of starting quarterbacks in the National Football League',
 'NFL International Series',
 '2008 San Francisco 49ers season',
 'Colin in Black & White',
 'Jim Harbaugh',
 'List of Denver Broncos starting quarterbacks',
 'U.S. national anthem kneeling protests',
 'San Francisco 49ers',
 'Alex Smith',
 'List of Cincinnati Bengals starting quarterbacks',
 'Nessa',
 'List of Dallas Cowboys starting quarterbacks',
 'Colin Kaepernick',
 '2012 NFL season',
 '1988 San Francisco 49ers season',
 '2013 NFL season',
 'National League (Eng

In [ ]:
def repl(doc):
	doc = doc.replace(' ', '_')
	doc = doc.replace('(', '-LRB-')
	doc = doc.replace(')', '-RRB-')
	doc = doc.replace(':', '-COLON-')
	return doc

In [ ]:
with open('IDs.txt', 'r') as f:
		doc_ids = f.read()

In [ ]:
def extract_from_dump(entities):
	entities = [repl(entity) for entity in entities]
	docs = []
	for entity in entities:
		if entity in doc_ids:
			docs.append(entity)
	return docs

In [ ]:
dump_docs = extract_from_dump(noun_phrases)
dump_docs

['the_National_Football_League', 'Colin_Kaepernick']

In [ ]:
in_path = '/content/shared_task_dev.jsonl'

In [ ]:
def filter(documents, claim):
	final_documents = []
	stemmer = PorterStemmer() 
	claim_tokens = word_tokenize(claim) 
	claim_tokens = [stemmer.stem(token.lower()) for token in claim_tokens]
	claim_tokens = list(set(claim_tokens))

	for doc in documents:
		left = doc.find('(')
		right = doc.find(')')
		if (left != -1 and right != -1):
			new_doc = doc[:left] + doc[right+1:]
		else:
			new_doc = doc
		doc_tokens = word_tokenize(new_doc) 
		doc_tokens = [stemmer.stem(token.lower()) for token in doc_tokens]
		doc_tokens = list(set(doc_tokens))

		include_flag = True
		for token in doc_tokens:
			if token not in claim_tokens:
				include_flag = False
				break	

		if include_flag:
			doc = repl(doc)
			doc = unicodedata.normalize('NFD', doc)
			final_documents.append(doc)
	return final_documents

In [ ]:
def doc_retriever(claim):
	# Step 1: Mention Extraction
	parse = predictor.predict(claim)
	root = parse['hierplane_tree']['root']

	noun_phrases = extract_noun_phrases(root)
	other_entities = extract_other_entities(root, claim)

	entities = []
	entities.extend(noun_phrases)
	entities.extend(other_entities)
	entities.append(claim)
	entities = list(set(entities))

	# Step 2: Candidate Article Search
	documents = wiki_search(claim, entities)
	dump_docs = extract_from_dump(noun_phrases)
	documents.extend(dump_docs)

	# Step 3: Candidate Filtering
	final_documents = filter(documents, claim)
	return final_documents



In [ ]:
print(doc_retriever(claim1))

['Colin_Kaepernick', 'National_League_-LRB-English_football-RRB-', 'National_Football_League']
